# ADCS Simulator and System Modeling / Selection Notebook 
Follow the readme to install dependencies and correctly configure you environment/machine to run the varios blocks.

In [1]:
%matplotlib widget
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
import lib.physics as physics
import lib.cubesat as cubesat
import lib.renderer as renderer

In [3]:
def vcircular(altitude):
    return math.sqrt(physics.Consts.mu / (altitude + physics.Consts.R_earth))

initial_state  = cubesat.State(physics.Consts.ISS_altitude + physics.Consts.R_earth, 0, 0, 0, vcircular(physics.Consts.ISS_altitude), 0) # x, y, z, dx, dy, dz
qubesat = cubesat.Cubesat(1, length=0.1, width=0.1, height=0.1) # mass in kg, length, width, height in m

planet  = physics.Planet(physics.Consts.M_earth, physics.Consts.R_earth) # mass in kg, radius in meters
# planet  = physics.Planet(10, 200) # mass in kg, radius in meters

In [4]:
class State:
    def __init__(self, x, y, z, dx, dy, dz):
        # nativelly contain the state as a cartesian state vector
        self.state_vector = np.array([x, y, z, dx, dy, dz])
        
    def get_cartesian_state_vector(self):
        """
        Structure of the state vector (units are m and m/s)
        x, y, z, dx, dy, dz
        """
        return self.state_vector
        
    def get_spherical_state_vector(self):
        """
        Structure of the earth referenced state vector (units are m, radians, and m/s)
        r (radius from the center of the plant), theta (latitude), phi (longitude), dr, dtheta, dphi
        """
        r = np.linalg.norm(self.state_vector[:3])
        phi = np.arctan2(self.state_vector[1], self.state_vector[0]) # latitude
        theta = np.arctan2(np.linalg.norm(self.state_vector[:2]), self.state_vector[2]) # longitude
        return np.array([r, theta, phi, 0, 0, 0])

    # Getter methods for cartesian coordinates 
    def get_x(self):
        return self.state_vector[0]
    
    def get_y(self):
        return self.state_vector[1]

    def get_z(self):
        return self.state_vector[2]
    
    def get_dx(self):
        return self.state_vector[3]
    
    def get_dy(self):
        return self.state_vector[4]
    
    def get_dz(self):
        return self.state_vector[5]
    
    # Getter methods for spherical coordinates
    def get_r(self):
        return self.get_spherical_state_vector()[0]
    
    def get_altitude(self, planet):
        return self.get_r() - planet.radius
    
    def get_theta(self):
        return self.get_spherical_state_vector()[1]
    
    def get_lat(self):
        return math.pi/2 - self.get_theta()

    def get_phi(self):
        return self.get_spherical_state_vector()[2]
    
    def get_lon(self):
        return self.get_phi()
    
    def get_dr(self):
        return self.get_spherical_state_vector()[3]
    
    def get_dtheta(self):
        return self.get_spherical_state_vector()[4]
    
    def get_dphi(self):
        return self.get_spherical_state_vector()[5]
    
def state_from_cartesian_state_vector(vector):
    return State(vector[0], vector[1], vector[2], vector[3], vector[4], vector[5])

def state_from_spherical_state_vector(vector):
    return State(vector[0], vector[1], vector[2], vector[3], vector[4], vector[5])

In [5]:
class Simulator:
    def __init__(self, cubesat, planet, state, dt=0.1):
        """
        time and step size in seconds
        """
        self.dt = dt
        self.cubesat = cubesat
        self.planet = planet
        self.elapsed_time = 0
        self.state = state
        # functions that take in a state and returns a force vector <Fx,Fy,Fz> acting on the COM of the cubesat
        self.forces = []
        # functions that take in a state and returns an acceleration vector <ax,ay,az> acting on the COM of the cubesat
        self.accelerations = []
        
        # The new state is provided by the following equation
        # xk = Axk-1 + Buk-1 + wk-1
        # u is the vector sum of all the accelerations
        
        # 3D state matrix - A
        self.A = np.array([[1, 0, 0, self.dt, 0, 0],
                          [0, 1, 0, 0, self.dt, 0],
                          [0, 0, 1, 0, 0, self.dt],
                          [0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 0, 1]]) 
        
        # 3D control matrix - B
        self.B = np.array([[1/2*self.dt**2, 0, 0],
                          [0, 1/2*self.dt**2, 0],
                          [0, 0, 1/2*self.dt**2],
                          [self.dt, 0, 0],
                          [0, self.dt, 0],
                          [0, 0, self.dt]])
        
    def step(self):
        # compute acceleration vector from gravity assuming points mass of the earth and satelite
        # [x,y,z]/|[x,y,z]| <- assumes point mass earth with center at (0,0,0)

        net_force = np.zeros(3)
        for forcer in self.forces:
            net_force += forcer(self.state, self.cubesat, self.planet)
            
        net_acc = np.zeros(3)
        for aaccelerator in self.accelerations:
            net_acc += aaccelerator(self.state, self.cubesat, self.planet)
            
        net_acc += net_force / self.cubesat.mass
        
        self.state = state_from_cartesian_state_vector(np.dot(self.A, self.state.get_cartesian_state_vector()) + np.dot(self.B, net_acc))
        self.elapsed_time += self.dt
        return (self.elapsed_time, self.state)
    
    def add_forcer(self, forcer):
        self.forces.append(forcer)
        
    def add_accelerator(self, accelerator):
        self.accelerations.append(accelerator)

In [70]:
class CartesianPlot:
    def __init__(self):
        self.is_3d = False
        pass
    
    def build(self, states, time_stamps, ax):
        x = [state.get_x() for state in states]
        y = [state.get_y() for state in states]
        z = [state.get_z() for state in states]
        ax.plot(time_stamps, x, color="red", label='x')
        ax.plot(time_stamps, y, color="green", label='y')
        ax.plot(time_stamps, z, color="blue", label='z')
        ax.set_title("Cartesian Coordinates Plot")
        ax.set_xlabel("time (s)")
        ax.set_ylabel("position xyz (m)")
        
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels)


from matplotlib.ticker import FuncFormatter, MultipleLocator
class SphericalPlot:
    def __init__(self):
        self.is_3d = False
        pass
    
    def build(self, states, time_stamps, ax):
        r = [state.get_r() for state in states]
        lon = [state.get_lon() for state in states]
        lat = [state.get_lat() for state in states]
        lns1 = ax.plot(time_stamps, lat, color="green", label='lat')
        lns2 = ax.plot(time_stamps, lon, color="blue", label='lon')
        ax.set_title("Spherical Coordinates Plot")
        ax.set_xlabel("time (s)")
        ax.set_ylabel("angle (radians)")
        ax.yaxis.set_major_locator(MultipleLocator(base=np.pi/2))
        
        ax2 = ax.twinx()
        lns3 = ax2.plot(time_stamps, r, color="red", label='radius')
        
        lns = lns1+lns2+lns3
        labels = [l.get_label() for l in lns]
        ax.legend(lns, labels, loc='upper center')
        
class OrbitalPlot3D:
    def __init__(self):
        self.is_3d = True
        pass
    
    def build(self, states, time_stamps, ax):
        x = [state.get_x() for state in states]
        y = [state.get_y() for state in states]
        z = [state.get_z() for state in states]
        ax.plot3D(x, y, z, 'red')
        ax.set_title("Orbital Plot 3D")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("z")
        
        count = 15 # keep 180 points along theta and phi
        # define a grid matching the map size, subsample along with pixels
        theta = np.linspace(0, np.pi, count)
        phi = np.linspace(0, 2*np.pi, count)
        theta,phi = np.meshgrid(theta, phi)
        R = physics.Consts.R_earth
        # sphere
        x = R * np.sin(theta) * np.cos(phi)
        y = R * np.sin(theta) * np.sin(phi)
        z = R * np.cos(theta)
        ax.plot_surface(x.T, y.T, z.T, cstride=1, rstride=1) # we've already pruned ourselves

        
class OrbitalPlot2D:
    def __init__(self):
        self.is_3d = False
        pass
    
    def build(self, states, time_stamps, ax):
        x = [state.get_x() for state in states]
        y = [state.get_y() for state in states]
        z = [state.get_z() for state in states]
        ax.plot(x, y, color="red", label='x')
        ax.set_title("Orbital Plot 2D")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.add_artist(plt.Circle((0,0), physics.Consts.R_earth, color='blue'))
        


class Renderer:
    def __init__(self):
        self.stop_condition = stop_condition
        self.time_stamps = []
        self.states = []
        self.plots = []
        
    def run(self, simulation, stop_condition):
        while not stop_condition(self.states, self.time_stamps):
            time, state = simulator.step()
            self.states.append(state)
            self.time_stamps.append(time)
            
    def add_plot(self, plot):
        self.plots.append(plot)
                    
    def render(self, figsize=(7,5), columns=3):
        fig = plt.figure(figsize=figsize)
        for i, plot in enumerate(self.plots):
            if plot.is_3d:
                ax = fig.add_subplot((len(self.plots)+1)//columns if not columns == 1 else len(self.plots), columns, i+1, projection='3d')
            else:
                ax = fig.add_subplot((len(self.plots)+1)//columns if not columns == 1 else len(self.plots), columns, i+1)
            plot.build(self.states, self.time_stamps, ax)
        plt.show()

In [73]:
def gravitational_forcer(state, cubesat, planet):
    return planet.get_gravitational_acceleration(state)


state = State(physics.Consts.ISS_altitude + physics.Consts.R_earth, 0, 0, 0, vcircular(physics.Consts.ISS_altitude) - 500, 0)
simulator = Simulator(qubesat, planet, state, 1)
simulator.add_accelerator(gravitational_forcer)

stop_condition = lambda states, times: len(states) > 10000
renderer = Renderer()
renderer.run(simulator, stop_condition)
plot1 = CartesianPlot()
renderer.add_plot(plot1)
plot2 = SphericalPlot()
renderer.add_plot(plot2)
plot3 = OrbitalPlot2D()
renderer.add_plot(plot3)
plot4 = OrbitalPlot3D()
renderer.add_plot(plot4)

renderer.render(figsize=(15,15), columns=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# References
[Passive Magentic Attitude Control fro Cubesat Spacecraft](https://lasp.colorado.edu/home/csswe/files/2012/06/Gerhardt_SSC10_PMAC.pdf)

[Comparison of Control Laws for Magnetic Detumbling](https://www.researchgate.net/publication/263008407_Comparison_of_Control_Laws_for_Magnetic_Detumbling)
[]()


https://docs.google.com/document/d/16QsLCta1YLF__5pwcChfcjYQvqa9x6MZi19Ohj2Yuls/edit

https://www.youtube.com/playlist?list=PL_D7_GvGz-v3mDQ9iR-cfjXsQf4DeR1_H

https://pypi.org/project/pyIGRF/

https://stackoverflow.com/questions/53074908/map-an-image-onto-a-sphere-and-plot-3d-trajectories


In [54]:
x = [9, 10, 11, 12]
x_t = [0.1, 0.2, 0.3, 0.1]
y = [5, 6, 7, 8]
z = [1, 2, 3, 4]

In [84]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(231)
ax = fig.add_subplot(232)
ax = fig.add_subplot(233)
ax = fig.add_subplot(234)
ax = fig.add_subplot(235)






# ax.plot(x_t, x, color="red", label='x', linestyle='--')
# ax.plot(y, color="green", label='y')
# ax.plot(z, color="blue", label='z')
# ax.set_title("Here")
# ax.set_xlabel("time (s)")
# ax.set_ylabel("position (m)")
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles, labels)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …